In [1]:
import os
os.chdir('../..')

In [2]:
from table_evaluator import load_data, TableEvaluator
# From https://github.com/Baukebrenninkmeijer/table-evaluator

from os import listdir
from os.path import isfile, join

import pandas as pd

In [3]:
dataset = 'Chicago'
synth_dir = '../synth_data/' + dataset + '/'
files = ['../data/' + dataset + '/data.csv']
labels = ['original']

for f in listdir(synth_dir):
    if isfile(join(synth_dir, f)):
        files.append(join(synth_dir, f))
        labels.append(f.split('.')[0])

In [4]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [5]:
df0 = pd.read_csv(files[0])
cat_cols = list(set(df0.columns) - set(continuous_columns))

In [8]:
names = ['TGAN', 'CTGAN', 'WGAN_WI_BO', 'WGAN_WI_NO']

for n in names:
    print(n + ":")
    real, fake = load_data(files[0], files[labels.index(n)])
    table_evaluator = TableEvaluator(real, fake, cat_cols)
    table_evaluator.evaluate(target_col='choice)
    print('--------------------------------------------------------')
    print('')

TGAN:

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1_real  f1_fake  jaccard_similarity
index                                                                    
LogisticRegression_real_testset       0.7653   0.2175              0.0738
LogisticRegression_fake_testset       0.1791   0.7591              0.0420
RandomForestClassifier_real_testset   0.8718   0.3131              0.1723
RandomForestClassifier_fake_testset   0.2665   0.7602              0.1244
DecisionTreeClassifier_real_testset   0.8191   0.2353              0.1347
DecisionTreeClassifier_fake_testset   0.1664   0.6582              0.0941
MLPClassifier_real_testset            0.8049   0.2891              0.1569
MLPClassifier_fake_testset            0.2506   0.7719              0.1049

Miscellaneous results:
                                           Result
Column Correlation Distance RMSE           0.0147
Column Correlation distance MAE            0.0